# Take Home Project: Wrangling coal mine data

* This task is an example of the kind of work we do to make public energy data usable for analysis.
* We want to be able to explore your general approach together and see how you think about these kinds of problems.
* **Spend 2-4 hours working on it.** This doesn't have to happen all at once. We want you to have time to play with the data, step away from it to think, and then come back to it again.
* Feel free to use whatever documentation or online resources you would normally consult while working on a data wrangling problem.
* Feel free to use additional 3rd party libraries if you want to.  You should be able to install them from within the notebook using `!pip install packagename` or `!conda install packagename`

## Email us your notebook within a week.
* Send it to [hello@catalyst.coop](mailto:hello@catalyst.coop) (normally we'd have you make a PR but... we don't want everyone looking at each others solutions)
* We'll review your notebook and if it looks good, we'll reach out to schedule a longer conversation about it, and another technical interview.

## Some questions to keep in mind:
* What assumptions are you making about the data?
* Is the raw data well structured?
* How will you test whether / when those assumptions are valid?
* How would you / did you deal with the data that don’t conform to those assumptions?
* If there are records which can’t be reasonably cleaned automatically, but were high value in an advocacy context, how would you integrate manual cleaning into the automated process so that the manual effort is captured, and can be incrementally improved over time?
* What expectations do you have about the output data?
* What kind of data validation checks would you design to make sure that the output meets your expectations? These could be either integrated into the table transformation process, or run on the final output.
* How do you decide when data isn’t recoverable?
* How will you evaluate the completeness of the data that you’ve been able to extract?
* What kind of queries are you trying to make easy with the structure of the output data?
* What parts of this process might make sense to generalize / abstract for re-use in extracting, cleaning, and reorganizing data from other tables?

## Background on coal mine data
* Mine data is helpful for understanding where power plants are purchasing fuel from and where new renewable energy can be built. This data is espesially relevant now that the Inflation Reduction Act (IRA) provides tax benefits for clean energy projects in coal communities. You can read more about "energy and coal communitites" [here](https://www.resources.org/common-resources/what-is-an-energy-community/), though not required to answer this interview question. 
* The PUDL database contains one table about coal mines from EIA 923 Form called [`coalmine_eia923`](https://catalystcoop-pudl.readthedocs.io/en/dev/data_dictionaries/pudl_db.html#fuel-receipts-costs-eia923). This table is extracted from another table that contains fuel contract purchases and costs called [`fuel_receipts_costs_eia923`](https://catalystcoop-pudl.readthedocs.io/en/dev/data_dictionaries/pudl_db.html#fuel-receipts-costs-eia923).
* The Mine Safety and Health Administration [provides a more complete set of mine data](https://arlweb.msha.gov/OpenGovernmentData/OGIMSHA.asp) that we haven't integrated into PUDL yet. MSHA dataset 13, aptly named ["Mines Data Set"](https://arlweb.msha.gov/OpenGovernmentData/DataSets/Mines.zip), contains detailed information about mines that `coalmine_eia923` does not. **The goal of this interview question is to extract the MSHA Mines Data Set and integrate it with the `coalmine_eia923` table.**

## Part A: Extract and Clean the MSHA Mine Data Set
Design a function or class that can extract the MSHA Mines Data Set. Clean the columns you think are required to defined a clear structure for the data. You can also clean columns you think would be helpful from an advocacy or research perspective though you don't need to clean every column. 

### Some questions to keep in mind
* What is the structure of the data? What information do you expect to be unique?
* If you were integrating this dataset into a database, what information could be noramlized into separate tables?
* How would you clean the most relevant fields for integrating with the `coalmine_eia923` table?

### Some hints
* The MSHA dataset has a [Definitions File](https://arlweb.msha.gov/OpenGovernmentData/DataSets/Mines_Definition_File.txt) with column type and description information. 
* Use latin encoding to extract the .txt files.

## Part B: Integrate the MSHA Mines Data Set with the `coalmine_eia923` table
The goal here is to integrate the useful information in the MSHA Mines dataset with the the `coalmine_eia923` table. Both datasets contain identifiers for mines called MSHA IDs. Unfortantely, the `coalmine_eia923` contains duplicate and missing MSHA ids. **Attempt to resolve the duplicates and impute the missing MSHA ids in the `coalmine_eia923` table so it can be joined with the MSHA mines dataset.**

You're welcome to use 3rd party libraries for this porition. Your solution doesn't need to be perfect but you should describe the limitations of your solutions and potential improvements.

### Some questions to keep in mind
* Which fields are the most reliable?
* Which fields are most helpful for identifying unique records?

### Some hints
* Sometimes, the original form respondents have to fill out can be helpful in understanding data quality issues due to data entry. [Here](https://www.eia.gov/Survey/#eia-923) is the EIA 923 Form.

# Set up access to the PUDL database
* You can download a copy of our PUDL SQLite DB from: https://data.catalyst.coop/pudl.db
* Substitute the path to that file on your system below:

In [ ]:
import sqlalchemy as sa
import pandas as pd

PUDL_DB_PATH = "/path/to/your/copy/of/pudl.db"

pudl_engine = sa.create_engine(f"sqlite:///{PUDL_DB_PATH}")

In [ ]:
# Read the coalmine_eia923 table:
coalmine_eia923 = pd.read_sql("coalmine_eia923", pudl_engine)
coalmine_eia923.info()